**Оглавление:**
1. [Изучение данных](#step1)  
2. [Предобработка данных](#step2) 
3. [Исследование заёмщиков](#step3)
4. [Общий вывод](#step4)


<a id="step1"></a>
## Изучение данных

Импортируем необходимые библиотеки:

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

ModuleNotFoundError: No module named 'pymystem3'

Прочитаем файл с данными:

In [ ]:
df = pd.read_csv('datasets/data.csv')

Получим первые 10 строк таблицы и общую информацию о данных:

In [ ]:
display(df.head(10), df.info())

В таблице имеем 21525 строк и 12 столбцов. В 2 столбцах тип данных `float`, в 5 — `int`, и еще в 5  — `object`.

Количество значений в столбцах `days_employed` и `total_income` отличается от количества значений в других столбцах, значит, в этих стобцах есть пропущенные значения.

В столбце `days_employed` встречаются данные с отрицательным количеством дней трудового стажа.

В столбце `education` данные отличаются по регистру.

В таблице встречаются столбцы с категоризированными данными (пары `education`-`education_id` и  `family_status`-`family_status_id`), которые при необходимости можно выделить в отдельные словари.

В столбце `purpose` данные представлены в произвольном виде, например, расходы на свадьбу отражены как "сыграть свадьбу" и "на проведение свадьбы", а операции с недвижимостью как "покупка жилья", "операции с жильем" или "покупка жилья для семьи".

Отметим, что наиболее важными для нашего исследования являются следующие столбцы: `debt`, `children`, `family_status`/`family_status_id`, `total_income` и `purpose`. На корректности данных в них следует обратить особое внимание.

**Вывод**

В каждой строке таблицы представлены данные о потенциальном заёмщике (его пол, возраст, семейное положение и наличие/отсутствие детей, уровень образования и ежемесячного дохода, тип занятости и трудовой стаж, наличие опыта задолженности по кредиту и цель получения кредита).

Предварительно можно утверждать, что данных достаточно для ответа на поставленные заказчиком вопросы. Но в данных встречаются пропуски и артефакты, некоторые данные заполнены без учета регистра и не приведены к единому образцу. Также таблица загромождена категоризированными данными.

Для дальнейшей работы потребуется предобработка данных.

<a id="step2"></a>
## Предобработка данных

### Обработка пропусков

Посчитаем, сколько в таблице пропущенных значений:

In [ ]:
display(df.isna().sum())

Мы имеем одинаковое количество пропущенных значений в столбцах `days_employed` и `total_income`. 

Судя по тому, что количество пропусков в обеих колонках одинаковое, можем предположить, что пропуски имеются в одних и тех же строках. Проверим это предположение, отобразив таблицу с пропущенными значениями:

In [ ]:
display(df[df.isna().any(1)])

Предположение оказалось верным: мы имеем 2174 строки с пропусками в столбцах `days_employed` и `total_income`. Очевидной зависимости между этими и другими столбцами не наблюдается. Возможно, данные в этих столбцах являются рассчетными, и утеря/порча первичных данных, из которых рассчитываются общий трудовой стаж в днях и ежемесячный доход, повлекла за собой пропуски в этих столбцах. В учебном проекте восстановить эти данные мы не можем.

Данные из столбца `days_employed` не будут влиять на результаты исследования, поэтому смело можем заполнить пропуски нулями (не используем `'unknown'`, т.к. дальше будем меня формат в этом столбце). 

Применим для этого метод `fillna()`:

In [ ]:
df['days_employed'] = df['days_employed'].fillna(0)

Для дальнейшего исследования нам важны значения в столбце `total_income`. Ежемесячный доход - количественная переменная, поэтому для оценки типичного значения выборки можно использовать среднее арифметическое или медиану. Поскольку мы имеем дело с показателем ежемесячного дохода, на мой взгляд, корректней взять медианное значение.

Предположим, что медианные значения дохода сильно различаются для разных типов занятости (возможно, пропуски нужно заполнить с учетом типа занятости). Выведем медианное значение дохода для разных типов занятости, а также посмотрим, какое количество данных у нас имеется по каждой из категорий:

In [ ]:
display(df.groupby('income_type')['total_income'].median())
display(df.groupby('income_type')['total_income'].count())

Видим, что медианное значение сильно отличается для разных категорий. Для корректного заполнения пропусков в столбце `total_income` заполним их медианным значением в соответствии с типом занятости:

In [ ]:
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda s: s.fillna(s.median()))

Еще раз подсчитаем, сколько в таблице пропущенных значений, чтобы убедиться, что мы устранили все пропуски:

In [ ]:
display(df.isna().sum())

**Вывод**

Мы обнаружили пропуски значений в двух колонках: `days_employed` и `total_income`. 
В столбце `days_employed` пропущенные значения мы заменили на ноль. В столбце `total_income` пропущенные значения заменены на медианное значение по этому столбцу.

### Замена типа данных

В таблице имеем 2 столбца (`total_income` и `days_employed`) с данными вещественного типа `float`. Веротяно, это рассчитанные величины, поэтому они имеют по 6 знаков после запятой, что не несет никакой дополнительной инофрмации, но сильно загромождает нашу таблицу с данными. Приведем данные в этих столбцах к целочисленному типу `int`: 

In [ ]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df

Проверим, что тип данных в колонке изменился:

In [ ]:
df.info()

**Вывод**

Мы заменили тип данных в столбцах `total_income` и `days_employed`, тем самым визуально разгрузив нашу таблицу.

### Обработка дубликатов

Посчитаем явные дубликаты в таблице:

In [ ]:
df.duplicated().sum()

Проверим, что из себя представляют эти дубликаты:

In [ ]:
df[df.duplicated()]

Из таблицы видим, что в дубликаты попали все строки с заполенными выше пропусками в колонках `days_employed` и `total_income`. Предположим, что из-за потери данных эти строки считываются как дубликаты, хотя на самом деле это всё данные о разных людях. В таком случае удаление этих строк повлечет за собой потерю данных, чего нам не хотелось бы. Оставим эти строки для дальнейшего исследования. 

Проанализируем важный для нашего исследования столбец `children	` при помощи метода `value_counts()`:

In [ ]:
df['children'].value_counts()

Слегка неправдоподобными выглядят семьи с количеством детей, равным 20 и -1. Возможно, при вводе данных произошла опечатка (но для опечатки это слишком много раз, и почему опечались столько раз именно в написании 1 и 2?), а может это просто мусорные данные, наверняка мы не знаем. Поскольку мы имеем дело с количественной переменной, заменим при помощи цикла `for` и условной конструкции `if` эти данные на медианное значение по столбцу, и проверим, что изменения призошли:

In [ ]:
for i in df['children']:
    if i == 20 or i == -1:
        df['children'] = df['children'].replace(i, df['children'].median())

df['children'].value_counts()

**Вывод**

Методом `duplicated().sum()` мы обнаружили дубликаты, но, проанализировав данные, приняли решение не удалять их.
Также методом `value_counts()` мы обнаружили неправдоподобные данные в столбце `children` и заменили их на медианное значение по этому столбцу.

Также на первом шаге мы отметили, что в столбце `education` данные отличаются по регистру. Но, поскольку эти данные не повлияют на дальнейшее исследование, приводить их к единому регистру мы не будем.

### Лемматизация

Составим список из уникальных значений в колонке `purpose`:

In [ ]:
unique_purpose = df['purpose'].unique()
unique_purpose

Применим лемматизацию, чтобы упорядочить данные в полученном списке `unique_purpose`. Импортируем библиотеку `pymystem3` с функцией лемматизации на русском языке, а также вызовем специальный контейнер `Counter` из модуля `collections`:

In [ ]:
m = Mystem() 

Посчитаем, какие слова встречаются в списке целей получения кредита. Для этого применим цикл `for` для лемматизации списка методом `lemmatize()` и посчитаем при помощи `Counter`, сколько раз слова встречаются в списке:

In [ ]:
lemmas = []

for j in unique_purpose:
    lemmas += m.lemmatize(j)
display(Counter(lemmas)) 


**Вывод**

В результате лемматизации столбца `purpose` можно выделить 5 ключевых слов: `свадьба`, `образование`, `автомобиль`, `недвижимость` и `жилье`. Последние 2 можно будет объединить в одну категорию. Эти данные нам понадобятся далее для проведения категоризации.

### Категоризация данных

Для оценки зависимости между уровнем дохода и возвратом кредита в срок нам необходимо разделить значения в колонке ежемесячного дохода `total_income` на несколько категорий. Для этого используем метод ` pd.qcut()` и получим на выходе 4 интервала (q=4):

In [ ]:
pd.qcut(df['total_income'], q=4)

Поделим значения в этом столбце на 4 категории:
* до 107798;
* от 107799 до 145017;
* от 145018 до 195543;
* от 195544.

Напишем функцию `income_group`, которая в качестве аргумента принимает значение ежемесячного дохода и возвращает номер категории. Используя метод `apply`, применим функцию к столбцу `total_income` и занесем данные в новый столбец df `total_income_group`. Проверим, что функция отработала верно.

In [ ]:
def income_group(income):
    if income <= 107798:
        return 0
    elif 107798 < income <= 145017:
        return 1
    elif 145017 < income <= 195543:
        return 2
    return 3

df['total_income_group'] = df['total_income'].apply(income_group)
df.head(10)

В разделе 2.1 мы выделили слова, которые определяют категории целей получения кредита.

Напишем функцию `purpose_group`, которая в качестве аргумента принимает строку, в которой указана цель получения кредита, и возвращает номер категории. Используя метод `apply`, применим функцию к столбцу `purpose` и занесем данные в новый столбец df `total_purpose_group`. Проверим, что функция отработала верно.

In [ ]:
def purpose_group(target):
    if 'свадьб' in target:
        return 0
    elif 'автомоб' in target:
        return 1
    elif 'образов' in target:
        return 2
    elif 'недвижим' or 'жиль' in target:
        return 3

df['total_purpose_group'] = df['purpose'].apply(purpose_group)
df.head(10)

**Вывод**

При помощи функций мы разделили на категории значения в столбцах `total_income` и `purpose` (по 4 категории на каждый столбец).

<a id="step3"></a>
## Исследование заёмщиков

### Зависимость между наличием детей и возвратом кредита в срок

Для выведения результатов создадим новый df `df_children_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index='children' – группируем данные по количеству детей, столбец `children`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [ ]:
df_children_pivot = df.pivot_table(index='children', columns='debt', values='purpose', aggfunc='count', margins=True)
df_children_pivot

Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [ ]:
df_children_pivot['debt_rate'] = df_children_pivot[1]/df_children_pivot['All']*100
df_children_pivot

**Вывод**

В результате мы увидели, что процент задолженностей незначительно, но все же увеличивается при наличии у клиента детей. 

Клиенты без детей являются должниками в 7.5% случаев. Далее процент должников растет при наличии 1 (9.2%) и 2 (9.4%) детей. Среди клиентов с тремя детьми должниками являются 8.2%, с четырьмя детьми - 9.8%. Среди клиентов с 5 детьми (таких в нашей выборке всего 9 человек) должники отсутствуют.

### Зависимость между семейным положением и возвратом кредита в срок

Сначала посмотрим, на какие группы по семейному положению разбиты наши данные. Для этого сгруппируем df по столбцу `family_status_id` и выведем уникальные значения по столбцу `family_status`:

In [ ]:
df.groupby('family_status_id')['family_status'].unique()

Для выведения результатов создадим новый df `df_fam_status_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index= ['family_status_id','family_status'] – группируем данные по id и семейному статусу, столбцы `family_status_id` и `family_status`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [ ]:
df_fam_status_pivot = df.pivot_table(
    index=['family_status_id','family_status']
    , columns='debt', values='purpose', aggfunc='count', margins=True)

df_fam_status_pivot

Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [ ]:
df_fam_status_pivot['debt_rate'] = df_fam_status_pivot[1]/df_fam_status_pivot['All']*100
df_fam_status_pivot

**Вывод**

В результате мы увидели, что число задолженностей незначительно, но все же меняется в зависимости от семейного статуса клиента.

Чаще имеют задолженности по кредитам клиенты с семейным статусом `гражданский брак` (задолженность в 9.3% случаев)  и `Не женат / не замужем` (задолженность в 9.7% случаев). Немного реже задолженности по кредитам имеют клиенты с семейным статусом `женат / замужем` (задолженность в 7.5% случаев)  и `в разводе` (задолженность в 7.1% случаев).  Реже всего имеют задолженность клиенты со статусом `вдовец / вдова` (задолженность в 6.6% случаев). 

### Зависимость между уровнем дохода и возвратом кредита в срок

Ранее мы разделили доходы клиентов на 4 категории.

Для выведения результатов создадим новый df `df_income_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index='total_income_group' – группируем данные по ранее выделенным категориям дохода, столбец `total_income_group`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [ ]:
df_income_pivot = df.pivot_table(index='total_income_group', columns='debt', values='purpose', aggfunc='count', margins=True)
df_income_pivot

Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [ ]:
df_income_pivot['debt_rate'] = df_income_pivot[1]/df_income_pivot['All']*100
df_income_pivot

**Вывод**

В результате мы увидели, что число задолженностей незначительно, но все же меняется в зависимости от ежемесячного дохода клиента. 

Чаще всего имеют задолженности по кредитам клиенты с ежемесячным доходом от 107799 до 145017 - задолженность в 8.8% случаев. Реже всего задолженности по кредитам у клиентов с ежемесячным доходом более 195544 - задолженность в 7.2% случаев. Чуть чаще имеют задолженность клиенты с ежемесячным доходом менее 107798 (задолженность в 7.9% случаев) и клиенты ежемесячным доходом от 145018 до 195543 (задолженность в 8.4% случаев).

### Зависимость цели кредита на его возврат в срок

Ранее мы разделили цели получения кредита на 4 категории.

Для выведения результатов создадим новый df `df_purpose_pivot`.

Воспользуемся методом `pivot_table` со следующими аргументами:
- index='total_purpose_group' – группируем данные по ранее выделенным категориям целей получения кредита, столбец `total_purpose_group`;
- columns='debt' – значения, по которым происходит группировка, столбец `debt`;
- values='purpose' – используем любой из столбцов df (в данном случае `purpose`), т.к. далее применяем `count`;
- aggfunc='count' – применяемая функция `count`;
- margins=True – получаем обобщенные значения по каждой группе.

In [ ]:
df_purpose_pivot = df.pivot_table(index='total_purpose_group', columns='debt', values='purpose', aggfunc='count', margins=True)
df_purpose_pivot

Добавим новый столбец `debt_rate`, в котором рассчитаем процент задолженностей в каждой группе, и выведем итоговую таблицу на экран:

In [ ]:
df_purpose_pivot['debt_rate'] = df_income_pivot[1]/df_income_pivot['All']*100
df_purpose_pivot

**Вывод**

В результате мы увидели, что число задолженностей незначительно, но все же меняется в зависимости от цели кредита. 

Чаще всего имеют задолженности по кредитам клиенты, цели кредита которых относятся к категориям `автомобиль` или `образование` - задолженность в 8.8% и 8.4% случаев соответственно. Реже задолженности по кредитам имеют клиенты, цели кредита которых относятся к категориям `свадьба` (задолженность в 7.9% случаев) или `недвижимость/жильё` (задолженность в 7.2% случаев).

<a id="step4"></a>
## Общий вывод

В первую очередь хотелось бы отметить, что по любой из выделенных нами категорий процент задолженностей составлял менее 10% (что, на мой взгляд, довольно неплохой показатель).

Также хочется отметить, что разница между показателями процента задолженности в каждой из катерогий очень незначительная и не превышает максимум 3% по категории.

Расположим клиентов в порядке возрастания вероятности задолженности по кредиту (т.е. чем больше число, тем больше вероятность, что клиент будет должником) в разных категориях:

* Наличие/отсутствие детей:
1. 5 детей.
2. Дети отсутствуют.
3. 3 детей.
4. 1 ребенок.
5. 2 детей.
6. 4 детей.

* Семейное положение:
1. Вдовец / вдова.
2. В разводе.
3. Женат / замужем.
4. Гражданский брак.
5. Не женат / не замужем.

* Ежемесячный доход:
1. Более 195544.
2. Менее 107798.
3. От 145018 до 195543.
4. От 107799 до 145017.

* Цель кредита:
1. Свадьба.
2. Недвижимость/жильё.
3. Образование.
4. Автомобиль.
